<a href="https://colab.research.google.com/github/pvnsekhar/odbcs_data_to_sqlite_vs_pickle/blob/main/data_push_to_sqlite_vs_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: load all files present in the folder to sqlite inmemory database

import sqlite3
import os
import pandas as pd

def load_files_to_sqlite(folder_path):
    """Loads all CSV and TXT files from a folder into an in-memory SQLite database.

    Args:
        folder_path: The path to the folder containing the files.
    """
    conn = sqlite3.connect(':memory:')  # Use in-memory database
    cursor = conn.cursor()

    for filename in os.listdir(folder_path):
        if filename.endswith(('housing_train.csv', '.txt')):  # Process CSV and TXT files
            filepath = os.path.join(folder_path, filename)
            try:
                if filename.endswith('.csv'):
                    df = pd.read_csv(filepath)
                else:  # Assume TXT file is space-separated
                    df = pd.read_csv(filepath, delim_whitespace=True)

                table_name = filename[:-4]  # Use filename as table name (without extension)
                df.to_sql(table_name, conn, if_exists='append', index=False) #Replace table if exists

                print(f"Successfully loaded '{filename}' into table '{table_name}'")
            except Exception as e:
                print(f"Error loading '{filename}': {e}")

    return conn

# Example usage (replace with your folder path)
folder_path = "sample_data"  # Example folder path
conn = load_files_to_sqlite(folder_path)

# Now you can query the database
cursor = conn.cursor()
# Example query (replace 'your_table_name' with the actual table name)
cursor.execute("SELECT * FROM california_housing_train")
results = cursor.fetchall()




Successfully loaded 'california_housing_train.csv' into table 'california_housing_train'


In [ ]:
import pickle
import io

buffer=io.BytesIO()
pickle.dump(results,buffer)

In [9]:
# prompt: create a table in sqlite using california_housing_train.csv file

# Assuming 'conn' from the previous code is still available

conn1 = sqlite3.connect(':memory:')  # Use in-memory database
cursor = conn1.cursor()

# Create a new table named 'housing_data' if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS housing_data (
        longitude REAL,
        latitude REAL,
        housing_median_age REAL,
        total_rooms REAL,
        total_bedrooms REAL,
        population REAL,
        households REAL,
        median_income REAL,
        median_house_value REAL
    )
''')

cursor.executemany("INSERT INTO housing_data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", results)


conn1.commit() # Commit the changes to make them persistent


In [11]:
# prompt: dump pickle data to a csv file

import csv

buffer.seek(0)  # Rewind the buffer to the beginning

# Now write the pickle data to a CSV file
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header (optional, but recommended)
    # Assuming results is a list of tuples/lists with the same structure as your table
    if results:
        header = ['longitude',
        'latitude',
        'housing_median_age',
        'total_rooms',
        'total_bedrooms',
        'population',
        'households',
        'median_income',
        'median_house_value']  # Get column names from the cursor
        writer.writerow(header)

    # Iterate through the pickled data and write each row to the CSV
    for row in pickle.load(buffer):
      writer.writerow(row)

# ... (rest of your existing code) ...